Transcription of Audio

In [ ]:
import sys
import time
import requests
import os
import pandas as pd
import numpy as np

In [ ]:
def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

In [ ]:
def get_url(token,filepath):
  '''
    Parameter:
      token: The API key
      data : The File Object to upload
    Return Value:
      url  : Url to uploaded file
  '''
  headers = {'authorization': token}
  response = requests.post('https://api.assemblyai.com/v2/upload',
                         headers=headers,
                         data=read_file(filepath))
  url = response.json()["upload_url"]
  print("Uploaded File and got temporary URL to file")
  return url

In [ ]:
def get_transcribe_id(token,url):
  '''
    Parameter:
      token: The API key
      url  : Url to uploaded file
    Return Value:
      id   : The transcribe id of the file
  '''
  endpoint = "https://api.assemblyai.com/v2/transcript"
  json = {
    "audio_url": url,"speaker_labels": True,"auto_highlights": True
  }
  headers = {
    "authorization": token,
    "content-type": "application/json"
  }
  response = requests.post(endpoint, json=json, headers=headers)
  id = response.json()['id']
  print("Made request and file is currently queued")
  return id

In [ ]:
def upload_file(token,filepath):
  '''
    Parameter: 
      filepath: The File Object to transcribe
    Return Value:
      token  : The API key
      transcribe_id: The ID of the file which is being transcribed
  '''
  
  # token = "398a8ab00a764f0092e93ff6a480a68f"
  file_url = get_url(token,filepath)
  transcribe_id = get_transcribe_id(token,file_url)
  return transcribe_id

In [ ]:
def get_text(token,transcribe_id):
  '''
    Parameter: 
      token: The API key
      transcribe_id: The ID of the file which is being 
    Return Value:
      result : The response object
  '''  
  endpoint = f"https://api.assemblyai.com/v2/transcript/{transcribe_id}"
  headers = {
    "authorization": token
  }
  result = requests.get(endpoint, headers=headers).json()
  return result

In [ ]:
def json_data_extraction(result,fname):

    audindex = pd.json_normalize(result['words'])
    audindex['fname'] = fname

    speakers = list(audindex.speaker)  # Change df to your dataframe name
    previous_speaker = 'A'
    l = len(speakers)
    i = 1
    speaker_seq_list = list()
    for index, new_speaker in enumerate(speakers):
        if index > 0:
            previous_speaker = speakers[index - 1]
        if new_speaker != previous_speaker:
            i += 1
        speaker_seq_list.append(i)
        # print(str(previous_speaker)+"  "+str(new_speaker)+"  "+str(i))
    audindex['seq'] = speaker_seq_list
    df = pd.DataFrame(audindex.groupby(['fname', 'speaker', 'seq']).agg(utter=('text', ' '.join), stime=('start', 'min'),etime=('end', 'max')))
    df.reset_index(inplace=True)
    df.sort_values(by=['stime'], inplace=True)

    df['stime'] = df.stime // 1000
    df['etime'] = df.etime // 1000
    df['seq'] = df.seq - 1
    df.rename(columns = {'speaker':'spcode'},inplace=True)
    # df.to_csv('tx_speaker_db.csv', mode='a', header=False, index=False)
    return df

In [ ]:
def generate_csv(audio_filepath):
  fname = os.path.splitext(os.path.basename(audio_filepath))[0]
  token =  "a2b89c113dc54485abe2692fc8e9ab30"
  tid = upload_file(token,audio_filepath)
  result = {}
  while result.get("status") != 'processing':
    result = get_text(token, tid)
  while result.get("status") != 'completed':
    result = get_text(token, tid)
  df = json_data_extraction(result,fname)
  return df




Audio Slicing 

In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pydub import AudioSegment
import os

In [ ]:
def crop_audio_csv(audio_filepath,output_path):
    # slicing/cropping audio files
    audio_df =(generate_csv(audio_filepath))
    fname = os.path.splitext(os.path.basename(audio_filepath))[0]
    for i in range(len(audio_df)):
        t1,t2 = audio_df.stime[i],audio_df.etime[i]
        t1 = t1 * 1000
        t2 = t2 * 1000
        newAudio = AudioSegment.from_wav(audio_filepath)
        newAudio = newAudio[t1:t2]
        newAudio.export(output_path+str(t1)+'_'+str(t2)+'_'+fname+".wav", format="wav")
    print('Cropped Successfully.')
    data = []
    os.chdir(output_path)
    output_path = os.getcwd()
    for r, d, f in os.walk(output_path):
       for file in f:
            if ".wav" in file:
              data.append((os.path.join(r,file)))

    df = pd.DataFrame(data,columns=['file'])
    from natsort import natsort_keygen
    df.sort_values(
    by="file",
    key=natsort_keygen(),inplace=True)
    df.reset_index(drop=True,inplace=True)
    df= pd.concat([df,generate_csv(audio_filepath)['utter']], axis=1)

    return df

In [ ]:
#!mkdir /content/crop_audio

In [ ]:
#crop_audio_csv('/content/YouAngry.wav','/content/crop_audio/')

SER Model

In [ ]:
!pip install transformers
!pip install tensorboardx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 2.2 MB/s 
     |████████████████████████████████| 6.6 MB 68.3 MB/s 
     |████████████████████████████████| 120 kB 91.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 2.2 MB/s 


In [ ]:
import os
import csv
import wave
import sys
import numpy as np
import pandas as pd
import glob
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
import os
from PIL import Image
from scipy.fftpack import fft
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import os
import chardet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
from transformers import BertModel, BertTokenizer
import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Audio Features 

In [ ]:
def audio2spectrogram(filepath):
    #fig = plt.figure(figsize=(5,5))
    samplerate, test_sound  = wavfile.read(filepath,mmap=True)
    _, spectrogram = log_specgram(test_sound, samplerate)
    #plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    return spectrogram
    
def audio2wave(filepath):
    fig = plt.figure(figsize=(5,5))
    samplerate, test_sound  = wavfile.read(filepath,mmap=True)
    plt.plot(test_sound)

In [ ]:
def log_specgram(audio, sample_rate, window_size=40,
                 step_size=20, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, _, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, np.log(spec.T.astype(np.float32) + eps)

In [ ]:
N_CHANNELS = 3
def get_3d_spec(Sxx_in, moments=None):
    if moments is not None:
        (base_mean, base_std, delta_mean, delta_std,
             delta2_mean, delta2_std) = moments
    else:
        base_mean, delta_mean, delta2_mean = (0, 0, 0)
        base_std, delta_std, delta2_std = (1, 1, 1)
    h, w = Sxx_in.shape
    right1 = np.concatenate([Sxx_in[:, 0].reshape((h, -1)), Sxx_in], axis=1)[:, :-1]
    delta = (Sxx_in - right1)[:, 1:]
    delta_pad = delta[:, 0].reshape((h, -1))
    delta = np.concatenate([delta_pad, delta], axis=1)
    right2 = np.concatenate([delta[:, 0].reshape((h, -1)), delta], axis=1)[:, :-1]
    delta2 = (delta - right2)[:, 1:]
    delta2_pad = delta2[:, 0].reshape((h, -1))
    delta2 = np.concatenate([delta2_pad, delta2], axis=1)
    base = (Sxx_in - base_mean) / base_std
    delta = (delta - delta_mean) / delta_std
    delta2 = (delta2 - delta2_mean) / delta2_std
    stacked = [arr.reshape((h, w, 1)) for arr in (base, delta, delta2)]
    return np.concatenate(stacked, axis=2)

AlexNet Model

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
#from .utils import load_state_dict_from_url
from torch.hub import load_state_dict_from_url

__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.num_classes=num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((12, 12))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        print('features',x.shape)
        return x
def alexnet(pretrained=False, progress=True, **kwargs):
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

Modified Alexnet 

In [ ]:
class ModifiedAlexNet(nn.Module):
    def __init__(self, num_classes=5):
        super(ModifiedAlexNet, self).__init__()
        self.num_classes=num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )
        self.Sigmoid = nn.Sigmoid

    def forward(self, x):
        x = self.features(x)
        x=torch.flatten(x, start_dim=2)
        x=torch.sum(x, dim=2)
        x=self.classifier(x)
        return x
   
def modifiedAlexNet(pretrained=False, progress=True, **kwargs):
    model_modified = ModifiedAlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model_modified.load_state_dict(state_dict)
    return model_modified

In [ ]:
def weight_init(m):
    if isinstance(m, torch.nn.Linear):
        print('init of linear is done')
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None: 
            torch.nn.init.xavier_uniform_(m.bias)

Combined Audio Text Model

In [ ]:
outputs_text= []
def hook_text(module, input, output):
    outputs_text.clear()
    outputs_text.append(output)
    return None

In [ ]:
outputs_audio= []
def hook_audio(module, input, output):
    outputs_audio.clear()
    outputs_audio.append(output)
    return None

In [ ]:
class CombinedAudioTextModel(nn.Module):
    def __init__(self, num_classes=5):
        super(CombinedAudioTextModel, self).__init__()
        self.num_classes=num_classes
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.text_model=torch.load('/content/drive/My Drive/model_text_best.pt')
        self.audio_model=torch.load('/content/drive/MyDrive/model_audio_best.pt')

        self.text_model.bert.pooler.register_forward_hook(hook_text)
        self.audio_model.features.register_forward_hook(hook_audio)

        for param in self.text_model.parameters():
          param.requires_grad = False
        for param in self.audio_model.parameters():
          param.requires_grad = False

        self.dropout = nn.Dropout(.5)
        self.linear = nn.Linear(1024, num_classes)

        self.Sigmoid = nn.Sigmoid

    def forward(self,text,audio):
        self.text_model(text)
        self.audio_model(audio)
        audio_embed=outputs_audio[0]
        text_embed=outputs_text[0]
        audio_embed=torch.flatten(audio_embed, start_dim=2)#a1,a2,a3......al{a of dim c} 
        audio_embed=torch.sum(audio_embed, dim=2)
        concat_embded=torch.cat((text_embed,audio_embed),1)
        x=self.dropout(concat_embded)
        x=self.linear(x)
        return x

In [ ]:
model=torch.load('/content/drive/MyDrive/Combined_model_audio_text.pt')
model.eval()
model.to('cpu')

CombinedAudioTextModel(
  (text_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features

In [ ]:
label_dict = {'ang': 0, 'hap': 1, 'neu': 3, 'sad': 2,'exc':4}
indextolabel = dict()
for key, value in label_dict.items():
  indextolabel[value] = key

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def convert_audio(pathAudio):
  #print("File to be converted {}".format(pathAudio))
  y, sr = librosa.load(pathAudio, sr = 16000, mono=True)
  y = y * 32767 / max(0.01, np.max(np.abs(y)))
  new_pathAudio = pathAudio.split(".wav")[0] + "_converted.wav"
  #print("Converted file saved at {}".format(new_pathAudio))
  wavfile.write(new_pathAudio, sr, y.astype(np.int16))
  return new_pathAudio

Main Function For Predicting Emotion

In [ ]:
def predict_emotion(file_path,text):
  prob = None
  preds = None
  try:
    if audio2spectrogram(file_path).shape[1] == 0: 
      return {}
  except:
     file_path = convert_audio(file_path)
     filename=file_path.split('/')[-1].strip('.wav')
     spector=audio2spectrogram(file_path)
     spector=get_3d_spec(spector)
     npimg = np.transpose(spector,(2,0,1))
     input_tensor=torch.tensor(npimg)
     sprectrome = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
     input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
     with torch.no_grad():
        if (sprectrome.shape[2]>65):
          output = model(input_ids,sprectrome)
          m = nn.Softmax(dim=1)
          probs = m(output)
          preds = torch.argmax(probs)
          prob = torch.max(probs)
          #print("Predicted class is {} with {}% probability".format(indextolabel[preds.item()], round(prob.item() * 100,2)))
          #print()
          probs = probs.tolist()[0]
          emotion ={}
          for i in range(len(probs)):
            emotion[indextolabel[i]] = round(probs[i] * 100,2)         
          return emotion

In [ ]:
from tqdm import tqdm
tqdm.pandas()

Predict Emotion (CSV File)

In [ ]:
def predict_emotion_csv(audio_filepath,output_path):
  df1 = crop_audio_csv(audio_filepath,output_path)
  df1['emotions'] = df1.progress_apply(lambda x:predict_emotion((x['file']),x['utter']),axis=1 )
  return df1




In [ ]:
#predict_emotion_csv('/content/combine.wav','/content/crop_audio/')

Uploaded File and got temporary URL to file
Made request and file is currently queued
Cropped Successfully.
Uploaded File and got temporary URL to file
Made request and file is currently queued


  0%|          | 0/59 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 640 is greater than input length  = 2, using nperseg = 2
  .format(nperseg, input_length))
100%|██████████| 59/59 [00:11<00:00,  5.24it/s]


,file,utter,emotions
0,/content/crop_audio/0_9000_combine.wav,"Radix Web have, you know, amongst the best dev...","{'ang': 2.61, 'hap': 42.21, 'sad': 0.15, 'neu'..."
1,/content/crop_audio/9000_10000_combine.wav,Hi.,None
2,/content/crop_audio/10000_10000_combine.wav,We've had a really positive experience working...,{}
3,/content/crop_audio/11000_30000_combine.wav,"I think on both fronts, really good. I think f...","{'ang': 0.93, 'hap': 50.56, 'sad': 0.01, 'neu'..."
4,/content/crop_audio/30000_49000_combine.wav,"Look, it works pretty well. There's always goi...","{'ang': 4.43, 'hap': 22.85, 'sad': 0.28, 'neu'..."
5,/content/crop_audio/49000_55000_combine.wav,"Yes, absolutely.","{'ang': 18.31, 'hap': 5.38, 'sad': 1.75, 'neu'..."
6,/content/crop_audio/56000_77000_combine.wav,I think the most impressive thing would be Rad...,"{'ang': 0.04, 'hap': 90.11, 'sad': 0.01, 'neu'..."
7,/content/crop_audio/77000_91000_combine.wav,That's right.,"{'ang': 7.52, 'hap': 9.29, 'sad': 0.16, 'neu':..."
8,/content/crop_audio/91000_117000_combine.wav,"Yeah, look, I think the trust is built over ti...","{'ang': 4.83, 'hap': 28.18, 'sad': 0.04, 'neu'..."
9,/content/crop_audio/117000_119000_combine.wav,The environment splendid. I think there's clea...,"{'ang': 0.15, 'hap': 84.68, 'sad': 0.04, 'neu'..."
